In [2]:
import numpy as np
import scipy
import sympy as sp

import plotly.graph_objects as go
from qiskit.opflow import I, X, Y, Z, H

from qaoa import *

import qiskit.tools.jupyter
%qiskit_version_table

import pickle
import warnings
warnings.filterwarnings("ignore")

In [3]:
def graph_energy(res, energy, points = None):
    s = np.linspace(0, 2 * np.pi, res + 1)[:-1]
    
    fig = go.Figure(data=[
            go.Surface(x=s, y=s, z=energy, showscale=False,
                       colorscale='Viridis')])

    
    if points is not None:
        px, py, p_energy = points
        fig.add_trace(go.Scatter3d(x=px, y=py, z=p_energy,
                                   mode='markers'))

    fig.update_scenes(
        xaxis_title='Gamma',
        yaxis_title='Beta',
        zaxis_title='Energy'
    )

    fig.update_layout(title='p = 1 Energy Landscape',
                    autosize=False,
                    width=1000, height=700,
                    )
    fig.show()

In [4]:
"""
Schwinger
"""
select = {'N': 4, #qubits
          'g' : 1,  #coupling
          'm' : 1,  #bare mass
          'a' : 1, #lattice spacing
          'theta' : 0, #topological term
          'mixer_type' : 'Y', # type of mixer {'X', 'Y', 'XY'}
         }

model = QuantumModel('schwinger', 'Standard')


ham = model.make(select)
schwinger_target = ham.target
ham

driver:
4.0 * IIII
+ 2.0 * IIZZ
+ 1.0 * IZIZ
+ 1.0 * IZZI
+ 4.0 * IIIZ
- 1.0 * IIZI
+ 3.0 * IZII
- 2.0 * ZIII
mixer:
1.0 * IIIY
+ 1.0 * IIYI
+ 1.0 * IYII
+ 1.0 * YIII
target:
1.0 * IIII
+ 0.5 * IIZZ
+ 0.25 * IZIZ
+ 0.25 * IZZI
+ 1.0 * IIIZ
- 0.25 * IIZI
+ 0.75 * IZII
- 0.5 * ZIII
+ 0.25 * IIIY
+ 0.25 * IIYI
+ 0.25 * IYII
+ 0.25 * YIII

In [5]:
p = 1
res = 128

In [6]:
print("Making symqaoa")
symqaoa = QAOA_Sympy(ham, p)
symqaoa.make()

Making symqaoa


In [7]:
print("Probability Landscape")
sym_probabilities = symqaoa.probability_landscape(res)
print("Energy Landscape")
sym_energies = symqaoa.energy_landscape(res)
print("Done!")

Probability Landscape
Energy Landscape
Done!


In [8]:
graph_energy(res, sym_probabilities)

In [9]:
graph_energy(res, sym_energies)

In [10]:
def find_landscape_minimas(landscape, filter = True):
    less_right = landscape <= np.roll(landscape, 1 , 0)
    less_left = landscape <= np.roll(landscape, -1 , 0)
    less_gamma = np.logical_and(less_right, less_left)

    less_bottom = landscape <= np.roll(landscape, 1 , 1)
    less_top = landscape <= np.roll(landscape, -1 , 1)
    less_beta = np.logical_and(less_bottom, less_top)

    minimas = np.logical_and(less_gamma, less_beta)

    res = landscape.shape[0]
    s = np.linspace(0, 2 * np.pi, res + 1)[:-1]

    x, y = np.meshgrid([s], [s])
    min_x = x[minimas]
    min_y = y[minimas]
    min_energy = landscape[minimas]

    if filter:
        filtered = min_energy <= np.mean(min_energy)
        min_x = min_x[filtered]
        min_y = min_y[filtered]
        min_energy = min_energy[filtered]

    return np.vstack([min_x, min_y, min_energy])


probability_minimas = find_landscape_minimas(sym_probabilities)
energy_minimas = find_landscape_minimas(sym_energies)
prob_min_x, prob_min_y, min_probs = probability_minimas
energy_min_x, energy_min_y, min_energies = energy_minimas

In [11]:
graph_energy(res, sym_energies, energy_minimas)

In [12]:
graph_energy(res, sym_probabilities, probability_minimas)

In [13]:
minimas = []

for x, y in zip(energy_min_x, energy_min_y):
    minimas.append(scipy.optimize.minimize(fun = lambda x: symqaoa.energy(*x), 
                            x0 = [x, y] * p,
                            bounds = [[0, 2 * np.pi] for _ in range(2 * p)]))
    
avg_energy = 0
for minima in minimas:
    avg_energy += minima.fun 
avg_energy /= len(minimas)

minimas = [minima for minima in minimas if minima.fun < avg_energy]

In [14]:
[minima.fun for minima in minimas]

[-2.07964928701366,
 -2.0796492870136576,
 -2.0796492870136616,
 -2.0796492870136487]

In [15]:
class ParametersNode:
    def __init__(self, 
                 parameters,
                 energy):
        
        self.parameters = parameters
        self.energy = energy

        self.id = tuple(np.round(parameters, 4))

        self.p = len(parameters) // 2

        self.gammas = parameters[::2]
        self.betas = parameters[1::2]

        self.layer = None

        self.prev = {}
        self.next = {}

    def optimize_next(self, 
                      energy_fn, 
                      initial_parameters):
        
        next_p = len(initial_parameters) // 2
        
        next_min = scipy.optimize.minimize(
                                    fun = lambda x: energy_fn(*x), 
                                    x0 = initial_parameters,
                                    bounds = [[0, 2 * np.pi] for _ in range(next_p * 2)])
        
        next_parameters = next_min.x
        next_energy = next_min.fun
        next_node = ParametersNode(next_parameters,
                                   next_energy)
        
        return next_node

class NodeLayer:
    def __init__(self):
        self.nodes = {}
        self.num_nodes = 0

        self.prev = None
        self.next = None

    def optimize_next(self, energy_fn, parameter_fn):
        for node in self.nodes.values():
            initial_parameters = parameter_fn(node.parameters)
            
            new_node = node.optimize_next(energy_fn, initial_parameters)

            if self.next is None:
                self.next = NodeLayer()
                self.next.prev = self

            # if parameter already exists, throw away current node and reference to originally created node
            if new_node.id in self.next.nodes.keys():
                new_node = self.next.nodes[new_node.id]
            else:
                self.next.add_node(new_node)

            node.next[new_node.id] = new_node
            new_node.prev[node.id] = node

    def add_node(self, new_node: ParametersNode):
        self.nodes[new_node.id] = new_node
        self.num_nodes = len(self.nodes)

In [16]:
layer0 = NodeLayer()

for minima in minimas:
    node = ParametersNode(minima.x, minima.fun)
    layer0.add_node(node)

In [17]:
symqaoa = QAOA_Sympy(ham, 2)
symqaoa.make_expectation()

In [25]:
def perturb_parameters(perturbation):
    def parameter_fn(parameters):
        new_parameters = np.hstack([parameters, np.zeros(2)])
        return new_parameters + perturbation
    return parameter_fn


perturbations = [0, 0.1, -0.1, 1, -1]
for pgamma in perturbations:
    for pbeta in perturbations:
        parameter_fn = perturb_parameters(np.array([0, 0, pgamma, pbeta]))
        layer0.optimize_next(symqaoa.energy, parameter_fn)

In [38]:
def perturb_parameters(perturbation):
    def parameter_fn(parameters):
        new_parameters = np.hstack([parameters, parameters])
        return new_parameters + perturbation
    return parameter_fn


perturbations = [0, 0.1, -0.1, 1, -1]
for pgamma in perturbations:
    for pbeta in perturbations:
        parameter_fn = perturb_parameters(np.array([0, 0, pgamma, pbeta]))
        layer0.optimize_next(symqaoa.energy, parameter_fn)

In [57]:
layer1 = NodeLayer()


min_energies = []
for node in layer0.next.nodes.values():
    if node.energy < min([node.energy for node in layer0.nodes.values()]):
        min_energies.append(node.energy)

avg_energy = np.mean(min_energies)

for node in layer0.next.nodes.values():
    if node.energy <= avg_energy:
        print(node.energy)
        layer1.add_node(node)

-2.103329635662936
-2.10917443605389
-2.1091744360459814
-2.109174436046082
-2.109174436053809
-2.113208128105671
-2.1132081281036106
-2.1132081281018062
-2.1132081281071855
-2.1091744360532987
-2.109174436034019
-2.109174435981984
-2.109174436053697
-2.109174431628726


14

In [58]:
numqaoa = QAOA_Numpy(ham)

def parameter_fn(parameters):
    g1, b1, g2, b2 = parameters
    new_parameters = np.array([g1, b1, (g1 + g2) / 2 , (b1 + b2) / 2, g2, b2])
    return new_parameters


layer1.optimize_next(numqaoa.energy, parameter_fn)

In [112]:
layer2 = NodeLayer()

min_energies = []
for node in layer1.next.nodes.values():
    if node.energy < min([node.energy for node in layer1.nodes.values()]):
        min_energies.append(node.energy)

avg_energy = np.mean(min_energies)

for node in layer1.next.nodes.values():
    if node.energy <= avg_energy:
        print(node.energy)
        layer2.add_node(node)

-2.144298844242643


In [94]:
def interpolate_params(parameters):
    p = len(parameters) // 2
    gammas, betas = parameters[::2], parameters[1::2]

    x_fit = np.linspace(0, 1, p)
    x_poly = np.linspace(0, 1, p + 1)

    gamma_interp = np.interp(x_poly, x_fit, gammas)
    beta_interp = np.interp(x_poly, x_fit, betas)
    new_parameters = np.array([val for pair in zip(gamma_interp, beta_interp) for val in pair])
    return new_parameters

def fit_params(parameters):
    p = len(parameters) // 2
    gammas, betas = parameters[::2], parameters[1::2]

    x_fit = np.linspace(0, 1, p)
    x_poly = np.linspace(0, 1, p + 1)

    gamma_fit = np.poly1d(np.polyfit(x_fit, gammas, 2))
    beta_fit = np.poly1d(np.polyfit(x_fit, betas, 2))

    new_parameters = np.array([val for pair in zip(gamma_fit(x_poly), beta_fit(x_poly)) for val in pair])
    return new_parameters

layer2.optimize_next(numqaoa.energy, fit_params)
layer2.optimize_next(numqaoa.energy, interpolate_params)

In [97]:
min_energies = []
for node in layer2.next.nodes.values():
    if node.energy < min([node.energy for node in layer2.nodes.values()]):
        min_energies.append(node.energy)
        print(node.energy)

In [72]:
for node in layer2.nodes.values():
    print(node.parameters)
    print(node.gammas)
    print(node.betas)

[4.69338410e+00 5.44211641e+00 2.49729616e+00 3.26180425e+00
 2.57748066e-01 2.97515171e-04]
[4.6933841  2.49729616 0.25774807]
[5.44211641e+00 3.26180425e+00 2.97515171e-04]


In [88]:
import plotly.graph_objects as go

def plot_parameters(layer):
    fig = go.Figure()

    for i, node in enumerate(layer.nodes.values()):
        p = len(node.parameters) // 2

        x_fit = np.linspace(0, 1, p)
        gamma_fit = np.poly1d(np.polyfit(x_fit, node.gammas, 2))
        beta_fit = np.poly1d(np.polyfit(x_fit, node.betas, 2))

        x_poly = np.linspace(0, 1, p + 1)

        gamma_interp = np.interp(x_poly, x_fit, node.gammas)
        beta_interp = np.interp(x_poly, x_fit, node.betas)

        fig.add_trace(go.Scatter(x= x_poly, y=gamma_fit(x_poly), name=f"gamma fit {i}"))
        fig.add_trace(go.Scatter(x= x_poly, y=beta_fit(x_poly), name=f"beta fit {i}"))

        fig.add_trace(go.Scatter(x= x_poly, y=gamma_interp, name=f"gamma interp {i}"))
        fig.add_trace(go.Scatter(x= x_poly, y=beta_interp, name=f"beta interp {i}"))
        
        fig.add_trace(go.Scatter(x=[0, 0.5, 1], y=node.gammas, name=f"gammas {i}"))
        fig.add_trace(go.Scatter(x=[0, 0.5, 1], y=node.betas, name=f"betas {i}"))

    fig.show()
    
plot_parameters(layer2)

In [75]:
np.linspace(0, 1,3)

array([0. , 0.5, 1. ])

In [103]:
def closeness_to_ground(val):
    return 1 - (ham.eigen_values()[-1] - val) / (ham.eigen_values()[-1] - ham.eigen_values()[0])

closeness_to_ground(-2.1442988) * 100

0.18481538464456548

In [65]:
ham.eigen_values()[0]

-2.1559760067703464

In [36]:
with open('data', 'wb') as f:
    pickle.dump(layer0,f)

with open('data', 'rb') as f:
    y = pickle.load(f)